In [10]:
import netket as nk
import netket.experimental as nkx

L_x = 2
L_y = 2
N_up = N_down = 1
pbc = False

if N_up is None:
    hilb8 = nk.hilbert.SpinOrbitalFermions(
        n_orbitals=L_x * L_y, s=1/2# , n_fermions_per_spin=(N_up, N_down)
    )
else:
    hilb8 = nk.hilbert.SpinOrbitalFermions(
        n_orbitals=L_x * L_y, s=1/2, n_fermions_per_spin=(N_up, N_down)
    )

hi = hilb8

t = 1
U = 8


def c(site, sz):
    return nk.operator.fermion.destroy(hi, site, sz=sz)


def cdag(site, sz):
    return nk.operator.fermion.create(hi, site, sz=sz)


def nc(site, sz):
    return nk.operator.fermion.number(hi, site, sz=sz)


g = nk.graph.Grid(extent=[L_x, L_y], pbc=pbc)
g8 = g
up = +1
down = -1
ham = 0.0

for sz in (up, down):
    for u, v in g.edges():
        # Map tile-local indices to global indices
        ham += -t * cdag(u, sz) @ c(v, sz) - t * cdag(v, sz) @c(u, sz)
for u in g.nodes():
    # Only add Hubbard U term for sites inside tile 0
    ham += U * nc(u, up) @ nc(u, down)


H8 = ham # nkx.operator.ParticleNumberAndSpinConservingFermioperator2nd.from_fermionoperator2nd(ham)

eig_vals, eig_vecs = nk.exact.lanczos_ed(H8, k=1, compute_eigenvectors=True)
print(f"Ground state energy: {eig_vals[0]}")





import numpy as np
from openfermion import fermi_hubbard, jordan_wigner, QubitOperator
from scipy.sparse.linalg import expm_multiply
from scipy.sparse import csc_matrix
from openfermion.linalg import get_sparse_operator
from scipy.sparse.linalg import eigsh

# -----------------------------
# 1. Hubbard Hamiltonian
# -----------------------------
# L_x, L_y = 3, 3
# N_up = N_down = 2
n_orbitals = L_x * L_y
n_qubits = 2 * n_orbitals  # Spin up and down
t = 1.0
U = 8.0

hubbard = fermi_hubbard(L_x, L_y, tunneling=-t, coulomb=U, periodic=pbc)
qubit_op = jordan_wigner(hubbard)  # QubitOperator

H_sparse = csc_matrix(get_sparse_operator(qubit_op, n_qubits=n_qubits))

# Niedrigste Eigenwerte von H_sparse (vollständige Spektrumsberechnung ist zu groß)
k = 1
eigs, eigv = eigsh(H_sparse, k=k, which='SA', return_eigenvectors=True)
if N_up is not None:
    warning = "N_up, N_down not supported, without particle number conservation"
else:
    warning = ""
print("Lowest eigenvalues:", eigs, warning)

def count_up_down(bitstring, n_orbitals):
    n_up = 0
    n_dn = 0
    for site in range(n_orbitals):
        up_bit = (bitstring >> (2*site)) & 1
        dn_bit = (bitstring >> (2*site + 1)) & 1
        n_up += up_bit
        n_dn += dn_bit
    return n_up, n_dn

full_dim = 2**n_qubits
if N_up is not None:
    valid_indices = []
    for i in range(full_dim):
        n_up, n_dn = count_up_down(i, n_orbitals)
        if n_up == N_up and n_dn == N_down:
            valid_indices.append(i)

if N_up is not None:
    print(f"Number of states in sector N_up={N_up}, N_down={N_down}: {len(valid_indices)}")            
    H_sector = H_sparse[np.ix_(valid_indices, valid_indices)]
    e_sector = eigsh(H_sector, k=1, which='SA', return_eigenvectors=False)[0]
    print("Sector GS:", e_sector)



/home/detlef/anaconda3/envs/NetKet/lib/python3.12/site-packages/netket/operator/_fermion2nd/jax.py:557: UserWarning: 
Consider using `netket.experimental.operator.ParticleNumberAndSpinConservingFermioperator2nd` to reduce the number of connected elements and
considerably reduce the computational cost.
You can convert this operator by calling `netket.experimental.operator.ParticleNumberAndSpinConservingFermioperator2nd.from_fermionoperator2nd`.

  super()._setup(self)


Ground state energy: -3.207750943219355
Lowest eigenvalues: [-3.20775094] N_up, N_down not supported, without particle number conservation
Number of states in sector N_up=1, N_down=1: 16
Sector GS: -3.2077509432193576


In [11]:
import pennylane as qml

H_penny = qml.spin.fermi_hubbard('rectangle',[L_x, L_y], hopping=t, coulomb=U)
H_p_sparse = H_penny.sparse_matrix()
eig_vals, eig_vecs = np.linalg.eig(H_p_sparse.toarray())
print(f"PennyLane Ground state energy: {np.min(eig_vals)}")

PennyLane Ground state energy: (-3.2077509432193514+3.944304525754038e-31j)


In [92]:
import pennylane as qml
import numpy as np

def qubit_op_to_linear_combination(qubit_op):
    coeffs = []
    ops = []

    for term, coeff in qubit_op.terms.items():
        coeffs.append(float(np.real(coeff)))

        if len(term) == 0:
            ops.append(qml.Identity(wires=[]))
            continue

        paulis = []
        for wire, pauli in term:
            if pauli == "X":
                paulis.append(qml.PauliX(wire))
            elif pauli == "Y":
                paulis.append(qml.PauliY(wire))
            elif pauli == "Z":
                paulis.append(qml.PauliZ(wire))
            else:
                raise ValueError(pauli)

        ops.append(qml.prod(*paulis))

    return qml.ops.LinearCombination(coeffs, ops)
LCU_operator = qubit_op_to_linear_combination(qubit_op)

In [93]:
import numpy as np

# Find the first integer i such that 2^i >= len(LCU_operator.terms()[0])
num_terms = len(LCU_operator.terms()[0])
i = int(np.ceil(np.log2(num_terms)))
print(f"Number of terms: {num_terms}")
print(f"First integer i such that 2^i >= {num_terms}: {i}")
print(f"2^{i} = {2**i}")
num_terms = len(LCU_operator.terms()[0])
control = [n_qubits + i for i in range(int(np.ceil(np.log2(num_terms))))]



Number of terms: 18
First integer i such that 2^i >= 18: 5
2^5 = 32


In [94]:
eigv.shape, H_sparse.shape
eig_state = np.zeros((2**n_qubits, 2**len(control)), dtype=complex)
eig_state[:,0:1] = eigv
eig_state = eig_state.flatten()


@qml.qnode(qml.device("default.qubit"))
def circuit(lcu, control):
    qml.StatePrep(eig_state, wires=range(n_qubits + len(control)))
    qml.PrepSelPrep(lcu, control)
    return qml.state()

state = circuit(LCU_operator, control)
print(f"State after LCU block-encoding:\n {state}")

print("probability of control 0 state:", np.sum(np.abs(state.reshape(2**n_qubits,-1)[:,0])**2))

res_state = state.reshape(2**n_qubits, -1)[:, 0].copy()
res_state /= np.linalg.norm(res_state)
overlap = np.abs(np.vdot(eigv.flatten(), res_state))**2
print("Overlap with ground state:", overlap)

State after LCU block-encoding:
 [-5.38670491e-33+5.82346477e-33j -1.93937759e-17+4.51316212e-17j
 -4.36759858e-34+3.49407886e-33j ... -9.48433628e-19+6.07691806e-19j
  6.31962031e-34-1.09930142e-34j  4.51295275e-19-3.90695219e-19j]
probability of control 0 state: 0.003927741494737012
Overlap with ground state: 1.0


In [95]:
overlap = np.abs(np.vdot(eigv.flatten(), res_state))**2
print("Overlap with ground state:", overlap)
print("probability of control 0 state:", np.sum(np.abs(state.reshape(2**n_qubits,-1)[:,0])**2))


Overlap with ground state: 1.0
probability of control 0 state: 0.003927741494737012


In [96]:
def reflection_zero_state(ancilla_wires):
    k = len(ancilla_wires)
    qml.PauliX(ancilla_wires[-1])
    # global phase flip on |11...1>
    qml.ctrl(
        qml.PhaseShift,
        control=ancilla_wires[:-1],
        control_values=[0]*(k-1)
    )(np.pi, ancilla_wires[-1])
    qml.PauliX(ancilla_wires[-1])

    qml.GlobalPhase(np.pi) # debugging to easier read it


dev = qml.device("default.qubit")
@qml.qnode(dev)
def apply_reflection(state_index):
    qml.BasisState(np.array(list(np.binary_repr(state_index, 2)), dtype=int), wires=[0,1])
    reflection_zero_state([0,1])
    return qml.state()

for i in range(4):
    out = apply_reflection(i)
    print(f"Input |{i:02b}> → Output amplitudes:", out.real)


Input |00> → Output amplitudes: [1. 0. 0. 0.]
Input |01> → Output amplitudes: [ 0. -1.  0.  0.]
Input |10> → Output amplitudes: [ 0.  0. -1.  0.]
Input |11> → Output amplitudes: [ 0.  0.  0. -1.]


In [97]:
@qml.qnode(qml.device("default.qubit"))
def apply_reflection():
    qml.StatePrep(state, wires=range(n_qubits + len(control)))
    reflection_zero_state(control)
    return qml.state()

state_reflect = apply_reflection()
    


In [98]:
res_state = state_reflect.reshape(2**n_qubits, -1)[:, 0]
np.linalg.norm(res_state)

np.float64(0.06267169612143118)

In [99]:
res_state /= np.linalg.norm(res_state)
overlap = np.abs(np.vdot(eigv.flatten(), res_state))**2
print("Overlap with ground state after reflection:", overlap)

Overlap with ground state after reflection: 1.0


In [100]:
def OAA_step(lcu, ancilla_wires):
    qml.PrepSelPrep(lcu, ancilla_wires)
    reflection_zero_state(ancilla_wires)
    qml.adjoint(qml.PrepSelPrep(lcu, ancilla_wires))  # U†
    reflection_zero_state(ancilla_wires)


In [101]:
@qml.qnode(qml.device("default.qubit"))
def apply_oaa(lcu, k=3):
    qml.StatePrep(state, wires=range(n_qubits + len(control)))
    for _ in range(k):
        OAA_step(lcu, control)
    return qml.state()

state_oaa = apply_oaa(LCU_operator,10)
    


In [102]:
res_state = state_oaa.reshape(2**n_qubits, -1)[:, 0]
np.linalg.norm(res_state)

np.float64(0.9289413731570395)

In [103]:
res_state /= np.linalg.norm(res_state)
overlap = np.abs(np.vdot(eigv.flatten(), res_state))**2
print("Overlap with ground state after reflection:", overlap)

Overlap with ground state after reflection: 0.9999999999999998


In [104]:
@qml.qnode(qml.device("default.qubit"))
def circuit(lcu, control):
    #for w in range(n_qubits):
    #    qml.Hadamard(w)
    qml.StatePrep(eig_state, wires=range(n_qubits + len(control)))
    qml.PrepSelPrep(lcu, control)
    return qml.state()

state = circuit(LCU_operator, control)
print(f"State after LCU block-encoding:\n {state}")

print("probability of control 0 state:", np.sum(np.abs(state.reshape(2**n_qubits,-1)[:,0])**2))

res_state = state.reshape(2**n_qubits, -1)[:, 0].copy()
print(np.linalg.norm(res_state))
res_state /= np.linalg.norm(res_state)
overlap = np.abs(np.vdot(eigv.flatten(), res_state))**2
print("Overlap with ground state:", overlap)

State after LCU block-encoding:
 [-5.38670491e-33+5.82346477e-33j -1.93937759e-17+4.51316212e-17j
 -4.36759858e-34+3.49407886e-33j ... -9.48433628e-19+6.07691806e-19j
  6.31962031e-34-1.09930142e-34j  4.51295275e-19-3.90695219e-19j]
probability of control 0 state: 0.003927741494737012
0.06267169612143118
Overlap with ground state: 1.0


In [105]:
state_oaa = apply_oaa(LCU_operator,3)
state_oaa

array([-1.89645925e-32+3.33036707e-32j,  1.88021082e-17-4.37547402e-17j,
       -7.83154759e-33-5.13807396e-33j, ...,
        4.77234024e-19-3.62220735e-19j, -1.73110123e-19-3.88403095e-19j,
       -5.19701962e-19+3.22213917e-19j], shape=(2048,))

In [106]:
res_state = state_oaa.reshape(2**n_qubits, -1)[:, 0]
np.linalg.norm(res_state)

np.float64(0.3084507857128908)

In [107]:
res_state /= np.linalg.norm(res_state)
overlap = np.abs(np.vdot(eigv.flatten(), res_state))**2
print("Overlap with ground state after oaa:", overlap)

Overlap with ground state after oaa: 1.0


In [108]:
from openfermion import QubitOperator

beta = 0.01  # dein Faktor

# 1. Identity-Term
op_trotter = QubitOperator('', 1.0)  # '' steht für Identity

# 2. qubit_op multiplizieren und subtrahieren
for term, coeff in qubit_op.terms.items():
    op_trotter += QubitOperator(term, -beta * coeff)

op_trotter = qubit_op_to_linear_combination(op_trotter)

In [109]:
@qml.qnode(qml.device("default.qubit"))
def circuit(lcu, control):
    for w in range(n_qubits):
        qml.Hadamard(w)
    qml.PrepSelPrep(lcu, control)
    return qml.state()

state = circuit(op_trotter, control)
print(f"State after LCU block-encoding:\n {state}")

print("probability of control 0 state:", np.sum(np.abs(state.reshape(2**n_qubits,-1)[:,0])**2))

res_state = state.reshape(2**n_qubits, -1)[:, 0].copy()
res_state /= np.linalg.norm(res_state)
overlap = np.abs(np.vdot(eigv.flatten(), res_state))**2
print("Overlap with ground state:", overlap)

State after LCU block-encoding:
 [ 0.10775862+7.91797499e-19j -0.00323276-6.40782528e-19j
 -0.01683026+3.95914288e-19j ...  0.00235746-1.65158156e-19j
 -0.00014915-1.32886347e-20j  0.00075656+7.07474804e-20j]
probability of control 0 state: 0.6588882282996416
Overlap with ground state: 0.0017916101326846002


In [110]:
state_oaa = apply_oaa(op_trotter, 2)
state_oaa

array([-0.00294703-2.10858681e-17j,  0.00681193-1.76583431e-18j,
        0.03323708+6.83663019e-18j, ..., -0.00381576-1.43797525e-18j,
        0.00023974+2.58306290e-20j, -0.00093472-1.28203809e-20j],
      shape=(2048,))

In [111]:
res_state = state_oaa.reshape(2**n_qubits, -1)[:, 0]
np.linalg.norm(res_state)

np.float64(0.35347196160269945)

In [112]:
res_state /= np.linalg.norm(res_state)
overlap = np.abs(np.vdot(eigv.flatten(), res_state))**2
print("Overlap with ground state after reflection:", overlap)

Overlap with ground state after reflection: 5.732859988546372e-05


In [126]:
@qml.qnode(qml.device("default.qubit"))
def circuit(lcu, control, k=1):
    for w in range(n_qubits):
        qml.Hadamard(w)
    for _ in range(k):
        qml.RZ(np.pi, wires=n_qubits)  
        qml.PrepSelPrep(lcu, control)
    return qml.state()

state = circuit(op_trotter, control, 4)
print(f"State after LCU block-encoding:\n {state}")

print("probability of control 0 state:", np.sum(np.abs(state.reshape(2**n_qubits,-1)[:,0])**2))

res_state = state.reshape(2**n_qubits, -1)[:, 0].copy()
res_state /= np.linalg.norm(res_state)
overlap = np.abs(np.vdot(eigv.flatten(), res_state))**2
print("Overlap with ground state:", overlap)

State after LCU block-encoding:
 [ 0.06737066+3.13005224e-17j -0.04974753+7.80313692e-18j
  0.0038316 -6.44098659e-18j ... -0.00837695+2.14082761e-18j
  0.00055093+4.69218113e-19j -0.00210041-1.34144454e-18j]
probability of control 0 state: 0.38074049217140604
Overlap with ground state: 0.001717149131917828


In [114]:
OpTrotter_sparse = op_trotter.sparse_matrix()


In [115]:
for cc in range(10):
    res_state = OpTrotter_sparse @ res_state
    res_state /= np.linalg.norm(res_state)
    print(cc, np.vdot(res_state, H_sparse @ res_state))

0 (5.380442138506654+0j)
1 (4.831863126802691+0j)
2 (4.346460143984556+0j)
3 (3.9169216810250385-2.465190328815662e-32j)
4 (3.536613246907598+2.465190328815662e-32j)
5 (3.1996269346676325+0j)
6 (2.9007567579224376+0j)
7 (2.635438888810957+2.465190328815662e-32j)
8 (2.399679553263799+0j)
9 (2.189982751958949+0j)


In [116]:
e_min, _ = eigsh(H_sparse, k=1, which='SR')
e_max, _ = eigsh(H_sparse, k=1, which='LR')
e_min, e_max

(array([-1.75480749]), array([24.]))

In [117]:
import scipy
H_scale = (H_sparse - e_max[0]*scipy.sparse.eye(H_sparse.shape[0]))/(e_max-e_min)

In [118]:
E, _ = eigsh(H_scale,k=1, which='SR')
E

array([-1.])

In [119]:
from openfermion import QubitOperator

qubit_scaleop = qubit_op - e_max[0] * QubitOperator('')

In [120]:
qubit_scaleop /= e_max[0]-e_min[0]

In [121]:
Hsp = csc_matrix(get_sparse_operator(qubit_scaleop, n_qubits=n_qubits))

In [122]:
e_min, _ = eigsh(Hsp, k=1, which='SR')
e_max, _ = eigsh(Hsp, k=1, which='LR')
e_min, e_max

(array([-1.]), array([-3.09660314e-15]))

In [129]:
LCU_scale = qubit_op_to_linear_combination(qubit_scaleop)

In [167]:
def R0(wire):
    qml.PauliX(wire)
    qml.PauliZ(wire)
    qml.PauliX(wire)

@qml.qnode(qml.device("default.qubit"))
def circuit(lcu, ancilla_wires, k):
    for w in range(n_qubits):
        qml.Hadamard(w)

    R0(ancilla_wires[0])   # ⬅️ ZWINGEND

    for _ in range(k):
        qml.PrepSelPrep(lcu, ancilla_wires)          # U
        R0(ancilla_wires[0])
        qml.adjoint(qml.PrepSelPrep)(lcu, ancilla_wires)  # U†
        R0(ancilla_wires[0])

    return qml.state()
  

state = circuit(LCU_scale, control, 90)
print(f"State after LCU block-encoding:\n {state}")

print("probability of control 0 state:", np.sum(np.abs(state.reshape(2**n_qubits,-1)[:,0])**2))

res_state = state.reshape(2**n_qubits, -1)[:, 0].copy()
res_state /= np.linalg.norm(res_state)
overlap = np.abs(np.vdot(eigv.flatten(), res_state))**2
print("Overlap with ground state:", overlap)

print(np.vdot(res_state, H_sparse @ res_state))

State after LCU block-encoding:
 [-0.02308883-5.32556065e-18j  0.05332311+3.27984591e-17j
  0.01177055+4.45923876e-17j ...  0.00911772-2.39007779e-18j
 -0.00420701-1.27406965e-16j  0.00584377-1.06911065e-16j]
probability of control 0 state: 0.057666311659110736
Overlap with ground state: 9.388528884905481e-05
(6.348756341930177+0j)


In [168]:
import numpy as np

# Eigenwerte: -1 (GS), +0.5
H = np.array([[-1.0, 0.0],
              [ 0.0, 0.5]])

eigvals, eigvecs = np.linalg.eigh(H)
ground_state = eigvecs[:, 0]

In [169]:
alpha0 = -0.25
alpha1 = -0.75

alpha = abs(alpha0) + abs(alpha1)

# normiertes H'
H_scaled = (alpha0/alpha)*np.eye(2) + (alpha1/alpha)*np.array([[1,0],[0,-1]])

In [170]:
import pennylane as qml
from pennylane import numpy as pnp

coeffs = pnp.array([abs(alpha0), abs(alpha1)])
coeffs = coeffs / pnp.sum(coeffs)

def PREP(anc):
    theta = 2 * pnp.arccos(pnp.sqrt(coeffs[0]))
    qml.RY(theta, wires=anc)

In [171]:
def SELECT(anc, sys):
    # ancilla |0> → I
    # ancilla |1> → Z
    qml.ctrl(qml.PauliZ, control=anc)(wires=sys)

In [172]:
def U_block(anc, sys):
    PREP(anc)
    SELECT(anc, sys)
    qml.adjoint(PREP)(anc)

In [173]:
@qml.qnode(qml.device("default.qubit", wires=2))
def qsvt_iteration(k):
    anc = 0
    sys = 1

    # Startzustand
    qml.Hadamard(sys)
    R0(anc)  # initiale Reflexion (wichtig!)

    for _ in range(k):
        U_block(anc, sys)
        R0(anc)
        qml.adjoint(U_block)(anc, sys)
        R0(anc)

    return qml.state()

In [181]:
k = 100
state = qsvt_iteration(k)

# Postselection
state = state.reshape(2,2)
psi = state[0]                 # ancilla |0>
prob = np.linalg.norm(psi)**2
psi /= np.linalg.norm(psi)

overlap = abs(np.vdot(ground_state, psi))**2
energy = np.vdot(psi, H @ psi).real

print("Postselection prob:", prob)
print("Overlap GS:", overlap)
print("Energy:", energy)

Postselection prob: 0.6249999999999855
Overlap GS: 0.8000000000000183
Energy: -0.7000000000000273


In [179]:
eigvals

array([-1. ,  0.5])

In [180]:
np.linalg.eig(H_scaled)

EigResult(eigenvalues=array([-1. ,  0.5]), eigenvectors=array([[1., 0.],
       [0., 1.]]))

In [183]:
import pennylane as qml
from pennylane import numpy as np

# 2x2 Hamiltonian: Eigenwerte -1 (GS), +0.5
H = np.array([[-1.0, 0.0],
              [ 0.0, 0.5]])
eigvals, eigvecs = np.linalg.eigh(H)
ground_state = eigvecs[:, 0]

# LCU coefficients for H = sum_i alpha_i P_i
coeffs = np.array([0.25, 0.75])
paulis = [qml.Identity, qml.PauliZ]  # P0=I, P1=Z
alpha = np.sum(coeffs)
H_scaled = H / alpha  # eigenvalues in [-1,1]

n_qubits = 2  # ancilla + system

def PREP(anc):
    theta = 2 * np.arccos(np.sqrt(coeffs[0]/np.sum(coeffs)))
    qml.RY(theta, wires=anc)

def SELECT(anc, sys):
    qml.ctrl(qml.PauliZ, control=anc)(wires=sys)

def U_block(anc, sys):
    PREP(anc)
    SELECT(anc, sys)
    qml.adjoint(PREP)(anc)

def R0(anc):
    qml.PauliX(anc)
    qml.PauliZ(anc)
    qml.PauliX(anc)

@qml.qnode(qml.device("default.qubit", wires=n_qubits))
def qsvt_iteration(k):
    anc, sys = 0, 1
    # Startzustand: system Hadamard
    qml.Hadamard(sys)
    # initiale Reflexion
    R0(anc)
    for _ in range(k):
        U_block(anc, sys)
        R0(anc)
        qml.adjoint(U_block)(anc, sys)
        R0(anc)
    return qml.state()

# Iterationen
k = 1
state = qsvt_iteration(k)
state = state.reshape(2,2)
psi = state[0]  # Postselection auf ancilla |0>
prob = np.linalg.norm(psi)**2
psi /= np.linalg.norm(psi)

overlap = abs(np.vdot(ground_state, psi))**2
energy = np.vdot(psi, H @ psi).real

print("Postselection probability:", prob)
print("Overlap with ground state:", overlap)
print("Energy:", energy)

Postselection probability: 0.6249999999999998
Overlap with ground state: 0.7999999999999999
Energy: -0.7


In [184]:
import pennylane as qml
from pennylane import numpy as np

# 2x2 Hamiltonian: Eigenwerte -1 (GS), +0.5
H = np.array([[-1.0, 0.0],
              [ 0.0, 0.5]])
eigvals, eigvecs = np.linalg.eigh(H)
ground_state = eigvecs[:, 0]

# LCU coefficients and Pauli operators
coeffs = np.array([0.25, 0.75])
paulis = [qml.Identity, qml.PauliZ]
alpha = np.sum(coeffs)
H_scaled = H / alpha  # Eigenvalues in [-1,1]

n_qubits = 2  # 1 ancilla + 1 system

# PREP: bereitet Koeffizienten auf Ancilla
def PREP(anc):
    theta = 2 * np.arccos(np.sqrt(coeffs[0]/np.sum(coeffs)))
    qml.RY(theta, wires=anc)

# SELECT: wendet konditional die Pauli-Operatoren an
def SELECT(anc, sys):
    qml.ctrl(qml.PauliZ, control=anc)(wires=sys)

# Explizites Block-Encoding
def U_block(anc, sys):
    PREP(anc)
    SELECT(anc, sys)
    qml.adjoint(PREP)(anc)

# Reflexion auf Ancilla |0>
def R0(anc):
    qml.PauliX(anc)
    qml.PauliZ(anc)
    qml.PauliX(anc)

# QSVT Iteration: W^k = (R U† R U)^k
@qml.qnode(qml.device("default.qubit", wires=n_qubits))
def qsvt_iteration(k):
    anc, sys = 0, 1
    qml.Hadamard(sys)  # Startzustand
    R0(anc)            # initiale Reflexion
    for _ in range(k):
        U_block(anc, sys)
        R0(anc)
        qml.adjoint(U_block)(anc, sys)
        R0(anc)
    return qml.state()

# Iterationen
k = 20
state = qsvt_iteration(k)
state = state.reshape(2,2)
psi = state[0]  # Postselection auf Ancilla |0>
prob = np.linalg.norm(psi)**2
psi /= np.linalg.norm(psi)

overlap = abs(np.vdot(ground_state, psi))**2
energy = np.vdot(psi, H @ psi).real

print("Postselection probability:", prob)
print("Overlap with ground state:", overlap)
print("Energy:", energy)

Postselection probability: 0.6250000000000034
Overlap with ground state: 0.7999999999999952
Energy: -0.6999999999999929


In [188]:
import pennylane as qml
from pennylane import numpy as np

# 2x2 Hamiltonian: Eigenwerte -1 (GS), +0.5
H = np.array([[-1.0, 0.0],
              [ 0.0, 0.5]])
eigvals, eigvecs = np.linalg.eigh(H)
ground_state = eigvecs[:, 0]

# LCU-Koeffizienten
coeffs = np.array([0.25, 0.75])
alpha = np.sum(coeffs)
H_scaled = H / alpha

# Zwei Ancilla-Qubits + System
n_qubits = 3
ancilla_wires = [0,1]
sys_wire = 2

# PREP: Amplituden auf zwei Ancillas
def PREP(ancilla):
    theta = 2 * np.arccos(np.sqrt(coeffs[0]/np.sum(coeffs)))
    qml.RY(theta, wires=ancilla[0])
    qml.CNOT(wires=[ancilla[0], ancilla[1]])  # einfache Entanglement für LCU

# SELECT: konditionale Pauli-Operatoren
def SELECT(ancilla, sys):
    # ancilla |10> -> Z
    qml.ctrl(qml.PauliZ, control=ancilla[0])(wires=sys)
    qml.ctrl(qml.PauliZ, control=ancilla[1])(wires=sys)

# Block-Encoding
def U_block(ancilla, sys):
    PREP(ancilla)
    SELECT(ancilla, sys)
    qml.adjoint(PREP)(ancilla)

# Reflexion auf alle Ancillas |00>
def R0(ancilla):
    for a in ancilla:
        qml.PauliX(a)
    qml.MultiRZ(np.pi, wires=ancilla)
    for a in ancilla:
        qml.PauliX(a)

# QSVT Iteration
@qml.qnode(qml.device("default.qubit", wires=n_qubits))
def qsvt_iteration(k):
    qml.Hadamard(sys_wire)  # Startzustand
    R0(ancilla_wires)       # initiale Reflexion
    for _ in range(k):
        U_block(ancilla_wires, sys_wire)
        R0(ancilla_wires)
        qml.adjoint(U_block)(ancilla_wires, sys_wire)
        R0(ancilla_wires)
    return qml.state()

# Iterationen
k = 5
state = qsvt_iteration(k)
state = state.reshape([2]*n_qubits)
psi = state[0,0,:]  # Postselection auf |00> bei beiden Ancillas
prob = np.linalg.norm(psi)**2
psi /= np.linalg.norm(psi)

overlap = abs(np.vdot(ground_state, psi))**2
energy = np.vdot(psi, H @ psi).real

print("Postselection probability:", prob)
print("Overlap with ground state:", overlap)
print("Energy:", energy)

Postselection probability: 0.9999999999999996
Overlap with ground state: 0.4999999999999999
Energy: -0.24999999999999994


In [191]:
import numpy as np

# 2x2 Hamiltonian
H = np.array([[-1.0, 0.0],
              [ 0.0, 0.5]])
eigvals, eigvecs = np.linalg.eigh(H)
ground_state = eigvecs[:,0]

# LCU-Zerlegung: H = 0.25*I + 0.75*Z
coeffs = np.array([0.25, 0.75])
paulis = [np.eye(2), np.array([[1,0],[0,-1]])]
alpha = np.sum(coeffs)

# Skaliertes H für Block-Encoding
H_scaled = H / alpha

# Block-Encoding U as 4x4 matrix (1 ancilla)
# |0>⊗ψ -> |0> H_scaled |ψ> + |1> |⊥>
U = np.zeros((4,4))
# Basis ordering: |ancilla, system> = |00>,|01>,|10>,|11>
# ancilla=0 block = H_scaled
U[0:2,0:2] = H_scaled
# ancilla=1 block = sqrt(I - H_scaled^2)
I2 = np.eye(2)
U[2:4,0:2] = np.sqrt(I2 - H_scaled@H_scaled)
U[2:4,2:4] = np.eye(2)  # beliebig, Hauptsache U unitär
U[0:2,2:4] = np.zeros((2,2))

# Reflexion R0 = 2|0><0| - I auf ancilla
R0 = np.array([[1,0,0,0],
               [0,1,0,0],
               [0,0,-1,0],
               [0,0,0,-1]])

# Startzustand: |0>⊗|+>
psi0 = np.array([1,1,0,0])/np.sqrt(2)

# Iteration: W = R0 U† R0 U
k = 4
psi = psi0.copy()
for _ in range(k):
    psi = R0 @ U.conj().T @ R0 @ U @ psi

# Postselection auf ancilla |0>
psi_sys = psi[0:2]
prob = np.linalg.norm(psi_sys)**2
psi_sys /= np.linalg.norm(psi_sys)

overlap = np.abs(np.vdot(ground_state, psi_sys))**2
energy = np.vdot(psi_sys, H @ psi_sys).real

print("Postselection probability:", prob)
print("Overlap with ground state:", overlap)
print("Energy:", energy)

Postselection probability: 0.5019531249999999
Overlap with ground state: 0.9961089494163423
Energy: -0.9941634241245135


In [195]:
import pennylane as qml
from pennylane import numpy as np

# 2x2 Hamiltonian
H = np.array([[-1.0, 0.0],
              [ 0.0, 0.5]])
eigvals, eigvecs = np.linalg.eigh(H)
ground_state = eigvecs[:,0]

# LCU-Zerlegung: H = 0.25*I + 0.75*Z
coeffs = np.array([0.25, 0.75])
paulis = [qml.Identity, qml.PauliZ]
alpha = np.sum(coeffs)
H_scaled = H / alpha  # Werte in [-1,1]

# PennyLane Device: 1 Ancilla + 1 System
dev = qml.device("default.qubit", wires=2)

# Explizite Block-Encoding als Matrix (2x2 System, 1 Ancilla)
I2 = np.eye(2)
H_scaled_mat = H_scaled

# Reflexion auf |0> Ancilla
R0_mat = np.array([[1,0,0,0],
                   [0,1,0,0],
                   [0,0,-1,0],
                   [0,0,0,-1]])

# Startzustand |0>⊗|+>
psi0 = np.array([1,1,0,0])/np.sqrt(2)

# QNode, der eine Iteration ausführt
@qml.qnode(dev)
def step_iteration(state_vector):
    # Lade den aktuellen Zustand
    qml.StatePrep(state_vector, wires=[0,1])
    # Wir bauen U als Controlled-Z + sqrt(I-H^2)
    # Hier als Beispiel direkt als Diagonal-Operation
    # Hauptsache es ist unitär und reproduziert H_scaled auf |0>
    # Ancilla=0 block = H_scaled
    # Ancilla=1 block = sqrt(I - H_scaled^2)
    # U Matrix: 4x4
    U = np.zeros((4,4))
    U[0:2,0:2] = H_scaled_mat
    U[2:4,0:2] = np.sqrt(I2 - H_scaled_mat@H_scaled_mat)
    U[2:4,2:4] = np.eye(2)
    U[0:2,2:4] = np.zeros((2,2))
    
    # In PennyLane als Unitary anwenden
    qml.QubitUnitary(U, wires=[0,1])
    # Reflexion
    qml.QubitUnitary(R0_mat, wires=[0,1])
    # U†
    qml.QubitUnitary(U.conj().T, wires=[0,1])
    # Reflexion
    qml.QubitUnitary(R0_mat, wires=[0,1])
    
    return qml.state()

# Schrittweise Iteration
psi = psi0.copy()
k = 9
for i in range(k):
    psi = step_iteration(psi)

# Postselection Ancilla=0
psi_sys = psi[0:2]
prob = np.linalg.norm(psi_sys)**2
psi_sys /= np.linalg.norm(psi_sys)

overlap = np.abs(np.vdot(ground_state, psi_sys))**2
energy = np.vdot(psi_sys, H @ psi_sys).real

print("Postselection probability:", prob)
print("Overlap with ground state:", overlap)
print("Energy:", energy)

Postselection probability: 0.5000019073486328
Overlap with ground state: 0.9999961853172863
Energy: -0.9999942779759294


In [213]:
import pennylane as qml
from pennylane import numpy as np

# 2x2 Hamiltonian
H = np.array([[-1.0, 0.0],
              [ 0.0, 0.5]])
eigvals, eigvecs = np.linalg.eigh(H)
ground_state = eigvecs[:,0]

# 1 System + 1 Ancilla
n_qubits = 2
ancilla = 0
sys = 1

# Block-Encoding als 4x4 Matrix
I2 = np.eye(2)
H_scaled = H / np.sum([0.25,0.75])
U_block = np.zeros((4,4),dtype=complex)
# ancilla=0 block = H_scaled
U_block[0:2,0:2] = H_scaled
# ancilla=1 block = sqrt(I-H^2)
U_block[2:4,0:2] = np.sqrt(I2 - H_scaled@H_scaled)
U_block[2:4,2:4] = np.eye(2)
U_block[0:2,2:4] = np.zeros((2,2))

# Reflexion R0 = X Z X auf Ancilla
def R0_gate(wire):
    qml.PauliX(wire)
    qml.PauliZ(wire)
    qml.PauliX(wire)

# QNode: eine Iteration
dev = qml.device("default.qubit", wires=n_qubits)
@qml.qnode(dev)
def qsvt_step(state_vector):
    qml.StatePrep(state_vector, wires=[ancilla, sys])
    qml.QubitUnitary(U_block, wires=[ancilla, sys])
    R0_gate(ancilla)
    qml.QubitUnitary(U_block.conj().T, wires=[ancilla, sys])
    R0_gate(ancilla)
    return qml.state()

# Startzustand: |0>⊗|+>
psi0 = np.array([1/np.sqrt(2), 1/np.sqrt(2), 0, 0])

# Iteration
psi = psi0.copy()
k = 10
for i in range(k):
    psi = qsvt_step(psi)

# Postselection auf ancilla |0>
psi_sys = psi[0:2]
prob = np.linalg.norm(psi_sys)**2
psi_sys /= np.linalg.norm(psi_sys)

overlap = np.abs(np.vdot(ground_state, psi_sys))**2
energy = np.vdot(psi_sys, H @ psi_sys).real

print("Postselection probability:", prob)
print("Overlap with ground state:", overlap)
print("Energy:", energy)

Postselection probability: 0.5000004768371581
Overlap with ground state: 0.9999990463265929
Energy: -0.9999985694898894


In [202]:
import pennylane as qml
from pennylane import numpy as np

# 2x2 Hamiltonian: H = -1*|0><0| + 0.5*|1><1|
H = np.array([[-1.0, 0.0],
              [ 0.0, 0.5]])
eigvals, eigvecs = np.linalg.eigh(H)
ground_state = eigvecs[:,0]

# PennyLane Hamiltonian für PrepSelPrep
hamil = qml.Hamiltonian([ -1.0, 0.5 ], [qml.Projector([0],1), qml.Projector([1],1)])
# Alternativ: Pauli-Z Version
# hamil = qml.Hamiltonian([0.25, 0.75], [qml.Identity(1), qml.PauliZ(1)])

# Device: 1 System + 1 Ancilla
n_qubits = 2
ancilla = [0]
sys = [1]
dev = qml.device("default.qubit", wires=n_qubits)

# Reflexion auf Ancilla |0>
def R0_gate(wire):
    qml.PauliX(wire)
    qml.PauliZ(wire)
    qml.PauliX(wire)

# Startzustand: |0>⊗|+>
psi0 = np.array([1/np.sqrt(2), 1/np.sqrt(2), 0, 0])

# QNode: eine Iteration
@qml.qnode(dev)
def qsvt_step(state_vector):
    qml.StatePrep(state_vector, wires=[0,1])
    # U Block-Encoding über PrepSelPrep
    qml.PrepSelPrep(hamil, control=ancilla)
    R0_gate(ancilla[0])
    qml.adjoint(qml.PrepSelPrep)(hamil, control=ancilla)
    R0_gate(ancilla[0])
    return qml.state()

# Schrittweise Iteration
psi = psi0.copy()
k = 10
for i in range(k):
    psi = qsvt_step(psi)

# Postselection auf ancilla |0>
psi_sys = psi[0:2]
prob = np.linalg.norm(psi_sys)**2
psi_sys /= np.linalg.norm(psi_sys)

overlap = np.abs(np.vdot(ground_state, psi_sys))**2
energy = np.vdot(psi_sys, H @ psi_sys).real

print("Postselection probability:", prob)
print("Overlap with ground state:", overlap)
print("Energy:", energy)

Postselection probability: 2.0563158349924254e-19
Overlap with ground state: 0.7999999999999996
Energy: -0.6999999999999995


In [205]:
import pennylane as qml
from pennylane import numpy as np

# Hamiltonian
coeffs = [0.25, 0.75]
paulis = [qml.Identity(1), qml.PauliZ(1)]

n_qubits = 2
dev = qml.device("default.qubit", wires=n_qubits)

def prep_matrix_entry(basis_state):
    """Return output state when PrepSelPrep acts on a basis state."""
    @qml.qnode(dev)
    def circuit():
        qml.StatePrep(basis_state, wires=range(n_qubits))
        qml.PrepSelPrep(hamil, control=[0])
        return qml.state()
    return circuit()

# Erstelle Computational-Basis-Vektoren
basis_vectors = np.eye(4)
matrix = np.zeros((4,4), dtype=complex)

for i, b in enumerate(basis_vectors):
    out = prep_matrix_entry(b)
    matrix[:,i] = out

print("Matrix von PrepSelPrep:")
print(matrix)

Matrix von PrepSelPrep:
[[-0.66666667+8.16431199e-17j  0.        +0.00000000e+00j
   0.47140452-5.77304037e-17j  0.        +0.00000000e+00j]
 [ 0.        +0.00000000e+00j  0.33333333+0.00000000e+00j
   0.        +0.00000000e+00j  0.47140452+0.00000000e+00j]
 [ 0.47140452-5.77304037e-17j  0.        +0.00000000e+00j
  -0.33333333+4.08215600e-17j  0.        +0.00000000e+00j]
 [ 0.        +0.00000000e+00j  0.47140452+0.00000000e+00j
   0.        +0.00000000e+00j  0.66666667+0.00000000e+00j]]


In [214]:
U_block.conj().T@U_block

tensor([[1.       +0.j, 0.       +0.j, 0.       +0.j, 0.       +0.j],
        [0.       +0.j, 1.       +0.j, 0.       +0.j, 0.8660254+0.j],
        [0.       +0.j, 0.       +0.j, 1.       +0.j, 0.       +0.j],
        [0.       +0.j, 0.8660254+0.j, 0.       +0.j, 1.       +0.j]], requires_grad=True)

In [215]:
import numpy as np

H_scaled = np.array([[-1.0, 0.0],[0.0, 0.5]])
I2 = np.eye(2)

# Obere Hälfte: H_scaled
top = H_scaled

# Untere Hälfte: Finde V so dass [top; V] unitär ist
# QR-Faktorisation oder Gram-Schmidt
Q, R = np.linalg.qr(np.vstack([top, np.eye(2)]))
U_block = Q

# Prüfen
print("U_block unitär?", np.allclose(U_block.conj().T @ U_block, np.eye(4)))

ValueError: operands could not be broadcast together with shapes (2,2) (4,4) 

In [219]:
import numpy as np

# 2x2 Hamiltonian
H = np.array([[-1.0, 0.0],
              [ 0.0, 0.5]])
eigvals, eigvecs = np.linalg.eigh(H)
ground_state = eigvecs[:,0]

# Skaliere H auf [-1,1] (für Block-Encoding)
alpha = np.max(np.abs(np.linalg.eigvals(H)))
H_scaled = H / alpha
I2 = np.eye(2)

# Wir wollen eine 4x4 unitäre Matrix U_block bauen:
# obere 2x2 block = H_scaled
# untere 2x2 block so wählen, dass U_block unitär ist
top = H_scaled
# untere 2x2 block = sqrt(I - H_scaled^2), orthonormalisiert via QR
bottom = np.sqrt(I2 - H_scaled @ H_scaled)
# Stapeln
U_full = np.vstack([np.hstack([top, np.zeros((2,2))]),
                    np.hstack([bottom, np.eye(2)])])
# Orthonormalisieren über QR
Q, R = np.linalg.qr(U_full)
U_block = Q

# Prüfen Unitarität
print("U_block unitär:", np.allclose(U_block.conj().T @ U_block, np.eye(4)))

# Reflexion auf Ancilla |0>
R0 = np.diag([1,1,-1,-1])

# Startzustand |0>⊗|+>
psi = np.array([1/np.sqrt(2), 1/np.sqrt(2), 0, 0])

# QSVT-Iteration
k = 100
for i in range(k):
    psi = R0 @ U_block.conj().T @ R0 @ U_block @ psi

# Postselection Ancilla=0
psi_sys = psi[0:2]
prob = np.linalg.norm(psi_sys)**2
psi_sys /= np.linalg.norm(psi_sys)

overlap = np.abs(np.vdot(ground_state, psi_sys))**2
energy = np.vdot(psi_sys, H @ psi_sys).real

print("Postselection probability:", prob)
print("Overlap with ground state:", overlap)
print("Energy:", energy)

U_block unitär: True
Postselection probability: 0.6249999999999938
Overlap with ground state: 0.8000000000000079
Energy: -0.700000000000012


In [218]:
U_block.conj().T@U_block

array([[ 1.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00],
       [ 0.00000000e+00,  1.00000000e+00,  0.00000000e+00,
        -9.61481343e-17],
       [ 0.00000000e+00,  0.00000000e+00,  1.00000000e+00,
         0.00000000e+00],
       [ 0.00000000e+00, -9.61481343e-17,  0.00000000e+00,
         1.00000000e+00]])

In [220]:
import numpy as np

# Original 2x2 Hamiltonian
H = np.array([[-1.0, 0.0],
              [ 0.5, 0.0]]).T
eigvals, eigvecs = np.linalg.eigh(H)
ground_state = eigvecs[:,0]

# 1️⃣ Verschiebe H, sodass der niedrigste Eigenwert = 0
lambda_min = np.min(eigvals)
H_shifted = H - lambda_min * np.eye(2)

# 2️⃣ Skaliere auf [-1,1]
alpha = np.max(np.abs(np.linalg.eigvals(H_shifted)))
H_scaled = H_shifted / alpha
I2 = np.eye(2)

# 3️⃣ Erstelle unitäre U_block
top = H_scaled
bottom = np.sqrt(I2 - H_scaled @ H_scaled)
U_full = np.vstack([np.hstack([top, np.zeros((2,2))]),
                    np.hstack([bottom, np.eye(2)])])
Q, _ = np.linalg.qr(U_full)
U_block = Q

# Prüfen Unitarität
print("U_block unitär:", np.allclose(U_block.conj().T @ U_block, np.eye(4)))

# 4️⃣ Reflexion auf Ancilla |0>
R0 = np.diag([1,1,-1,-1])

# 5️⃣ Startzustand |0>⊗|+>
psi = np.array([1/np.sqrt(2), 1/np.sqrt(2), 0, 0])

# 6️⃣ QSVT-Iteration
k = 20
for step in range(1, k+1):
    psi = R0 @ U_block.conj().T @ R0 @ U_block @ psi

    # Postselection auf Ancilla=0
    psi_sys = psi[0:2]
    prob = np.linalg.norm(psi_sys)**2
    psi_sys /= np.linalg.norm(psi_sys)

    energy = np.vdot(psi_sys, H @ psi_sys).real
    overlap = np.abs(np.vdot(ground_state, psi_sys))**2
    print(f"Step {step:2d}: <H> = {energy:.6f}, overlap = {overlap:.6f}, P(ancilla=0) = {prob:.6f}")

U_block unitär: False
Step  1: <H> = nan, overlap = nan, P(ancilla=0) = nan
Step  2: <H> = nan, overlap = nan, P(ancilla=0) = nan
Step  3: <H> = nan, overlap = nan, P(ancilla=0) = nan
Step  4: <H> = nan, overlap = nan, P(ancilla=0) = nan
Step  5: <H> = nan, overlap = nan, P(ancilla=0) = nan
Step  6: <H> = nan, overlap = nan, P(ancilla=0) = nan
Step  7: <H> = nan, overlap = nan, P(ancilla=0) = nan
Step  8: <H> = nan, overlap = nan, P(ancilla=0) = nan
Step  9: <H> = nan, overlap = nan, P(ancilla=0) = nan
Step 10: <H> = nan, overlap = nan, P(ancilla=0) = nan
Step 11: <H> = nan, overlap = nan, P(ancilla=0) = nan
Step 12: <H> = nan, overlap = nan, P(ancilla=0) = nan
Step 13: <H> = nan, overlap = nan, P(ancilla=0) = nan
Step 14: <H> = nan, overlap = nan, P(ancilla=0) = nan
Step 15: <H> = nan, overlap = nan, P(ancilla=0) = nan
Step 16: <H> = nan, overlap = nan, P(ancilla=0) = nan
Step 17: <H> = nan, overlap = nan, P(ancilla=0) = nan
Step 18: <H> = nan, overlap = nan, P(ancilla=0) = nan
Step 1

/tmp/ipykernel_8376/1202858573.py:20: RuntimeWarning: invalid value encountered in sqrt
  bottom = np.sqrt(I2 - H_scaled @ H_scaled)


In [221]:
import numpy as np
from scipy.linalg import sqrtm

# Hamiltonian
H = np.array([[-1.0, 0.0],
              [0.5, 0.0]]).T
eigvals, eigvecs = np.linalg.eigh(H)
ground_state = eigvecs[:,0]

# Verschiebung
lambda_min = np.min(eigvals)
H_shifted = H - lambda_min * np.eye(2)

# Skalierung auf [-1,1]
alpha = np.max(np.abs(np.linalg.eigvals(H_shifted)))
H_scaled = H_shifted / alpha
I2 = np.eye(2)

# Matrix-Square-Root für Block-Encoding
sqrt_term = sqrtm(I2 - H_scaled @ H_scaled)

# U_block 4x4
U_block = np.block([[H_scaled, sqrt_term],
                    [sqrt_term, -H_scaled]])

# Prüfen Unitarität
print("U_block unitär:", np.allclose(U_block.conj().T @ U_block, np.eye(4)))

# Reflexion auf Ancilla |0>
R0 = np.diag([1,1,-1,-1])

# Startzustand |0>⊗|+>
psi = np.array([1/np.sqrt(2), 1/np.sqrt(2), 0, 0])

# Iteration
k = 20
for step in range(1, k+1):
    psi = R0 @ U_block.conj().T @ R0 @ U_block @ psi
    psi_sys = psi[0:2]
    prob = np.linalg.norm(psi_sys)**2
    psi_sys /= np.linalg.norm(psi_sys)
    energy = np.vdot(psi_sys, H @ psi_sys).real
    overlap = np.abs(np.vdot(ground_state, psi_sys))**2
    print(f"Step {step:2d}: <H> = {energy:.6f}, overlap = {overlap:.6f}, P(ancilla=0) = {prob:.6f}")

U_block unitär: False
Step  1: <H> = -0.250000, overlap = 0.100000, P(ancilla=0) = 1.250000
Step  2: <H> = -0.381712, overlap = 0.623913, P(ancilla=0) = 1.001742
Step  3: <H> = -0.269334, overlap = 0.111784, P(ancilla=0) = 0.583587
Step  4: <H> = -0.458786, overlap = 0.690027, P(ancilla=0) = 1.332968
Step  5: <H> = -0.154140, overlap = 0.047639, P(ancilla=0) = 0.100282
Step  6: <H> = -0.366187, overlap = 0.610056, P(ancilla=0) = 1.742323
Step  7: <H> = -1.038991, overlap = 0.871878, P(ancilla=0) = 0.100227
Step  8: <H> = -1.057659, overlap = 0.930553, P(ancilla=0) = 0.348356
Step  9: <H> = -0.957085, overlap = 0.736936, P(ancilla=0) = 3.525447
Step 10: <H> = -0.296876, overlap = 0.545824, P(ancilla=0) = 0.303232
Step 11: <H> = -0.507734, overlap = 0.282602, P(ancilla=0) = 3.380302
Step 12: <H> = 0.031306, overlap = 0.144484, P(ancilla=0) = 0.765278
Step 13: <H> = -0.192695, overlap = 0.067367, P(ancilla=0) = 2.291316
Step 14: <H> = 0.040504, overlap = 0.124666, P(ancilla=0) = 0.922781


/tmp/ipykernel_8376/3681987335.py:20: LinAlgWarning: Matrix is singular. The result might be inaccurate or the array might not have a square root.
  sqrt_term = sqrtm(I2 - H_scaled @ H_scaled)


In [222]:
import numpy as np

# 2x2 Hamiltonian
H = np.array([[-1.0, 0.0],
              [ 0.5, 0.0]]).T
eigvals, eigvecs = np.linalg.eigh(H)
ground_state = eigvecs[:,0]

# Startzustand: |0>⊗|+>
psi = np.array([1/np.sqrt(2), 1/np.sqrt(2), 0, 0])

# Wir simulieren ein "U_block" Filter direkt
# Obere 2x2 Block = H_scaled (normalisiert auf |λ_max| ≤ 1)
alpha = np.max(np.abs(eigvals))
H_scaled = H / alpha

# Für 1 System-1 Ancilla: konstruiere 4x4 Unitär über Gram-Schmidt
# Wir wollen U_block = [[H_scaled, ...], [..., ...]] unitär
# Einfach: komplette Spalten orthonormal erzeugen
U_block = np.eye(4)
U_block[0:2,0:2] = H_scaled
# Restliche Spalten via QR
Q, _ = np.linalg.qr(U_block)
U_block = Q

# Reflexion auf Ancilla |0>
R0 = np.diag([1,1,-1,-1])

# Iteration
k = 20
for step in range(1, k+1):
    psi = R0 @ U_block.conj().T @ R0 @ U_block @ psi

    # Postselection Ancilla=0
    psi_sys = psi[0:2]
    prob = np.linalg.norm(psi_sys)**2
    psi_sys /= np.linalg.norm(psi_sys)

    energy = np.vdot(psi_sys, H @ psi_sys).real
    overlap = np.abs(np.vdot(ground_state, psi_sys))**2
    print(f"Step {step:2d}: <H> = {energy:.6f}, overlap = {overlap:.6f}, P(ancilla=0) = {prob:.6f}")

Step  1: <H> = -0.250000, overlap = 0.500000, P(ancilla=0) = 1.000000
Step  2: <H> = -0.250000, overlap = 0.500000, P(ancilla=0) = 1.000000
Step  3: <H> = -0.250000, overlap = 0.500000, P(ancilla=0) = 1.000000
Step  4: <H> = -0.250000, overlap = 0.500000, P(ancilla=0) = 1.000000
Step  5: <H> = -0.250000, overlap = 0.500000, P(ancilla=0) = 1.000000
Step  6: <H> = -0.250000, overlap = 0.500000, P(ancilla=0) = 1.000000
Step  7: <H> = -0.250000, overlap = 0.500000, P(ancilla=0) = 1.000000
Step  8: <H> = -0.250000, overlap = 0.500000, P(ancilla=0) = 1.000000
Step  9: <H> = -0.250000, overlap = 0.500000, P(ancilla=0) = 1.000000
Step 10: <H> = -0.250000, overlap = 0.500000, P(ancilla=0) = 1.000000
Step 11: <H> = -0.250000, overlap = 0.500000, P(ancilla=0) = 1.000000
Step 12: <H> = -0.250000, overlap = 0.500000, P(ancilla=0) = 1.000000
Step 13: <H> = -0.250000, overlap = 0.500000, P(ancilla=0) = 1.000000
Step 14: <H> = -0.250000, overlap = 0.500000, P(ancilla=0) = 1.000000
Step 15: <H> = -0.25

In [223]:
import numpy as np

# 4x4 Hamiltonian (z.B. 2x2 Hubbard)
H = np.array([[ 1.0, 0.2, 0.0, 0.0],
              [ 0.2, 0.5, 0.1, 0.0],
              [ 0.0, 0.1, -0.5, 0.3],
              [ 0.0, 0.0, 0.3, -1.0]])
eigvals, eigvecs = np.linalg.eigh(H)
ground_state = eigvecs[:,0]

n = H.shape[0]
ancilla = n  # 1 Ancilla, System = n qubits
dim = n + ancilla

# 1️⃣ Startzustand |0>_a ⊗ |+>_s
psi = np.zeros(dim)
psi[:n] = 1/np.sqrt(n)  # superposition auf System

# 2️⃣ Skaliere H auf [-1,1] für QSVT
alpha = np.max(np.abs(eigvals))
H_scaled = H / alpha

# 3️⃣ Erstelle U_block über Gram-Schmidt
# Obere Hälfte = H_scaled, Rest = random und orthonormalisieren
U = np.zeros((dim, dim))
U[:n, :n] = H_scaled

# Restliche Spalten zufällig starten
for i in range(n, dim):
    col = np.random.randn(dim) + 1j*np.random.randn(dim)
    # Orthogonal zu bisherigen Spalten machen
    for j in range(i):
        col -= np.vdot(U[:,j], col) * U[:,j]
    # Normieren
    U[:,i] = col / np.linalg.norm(col)

# Unitär prüfen
print("U_block unitär:", np.allclose(U.conj().T @ U, np.eye(dim)))

# 4️⃣ Reflexion auf Ancilla=0
R0 = np.diag([1]*n + [-1]*ancilla)

# 5️⃣ QSVT-Iteration
k = 20
for step in range(1, k+1):
    psi = R0 @ U.conj().T @ R0 @ U @ psi
    psi_sys = psi[:n]
    prob = np.linalg.norm(psi_sys)**2
    psi_sys /= np.linalg.norm(psi_sys)
    energy = np.vdot(psi_sys, H @ psi_sys).real
    overlap = np.abs(np.vdot(ground_state, psi_sys))**2
    print(f"Step {step:2d}: <H> = {energy:.6f}, overlap = {overlap:.6f}, P(ancilla=0) = {prob:.6f}")

U_block unitär: False
Step  1: <H> = 0.701914, overlap = 0.157840, P(ancilla=0) = 0.397644
Step  2: <H> = 0.631565, overlap = 0.198461, P(ancilla=0) = 0.784615
Step  3: <H> = 0.542515, overlap = 0.238769, P(ancilla=0) = 0.783370
Step  4: <H> = 0.443327, overlap = 0.283564, P(ancilla=0) = 0.779567
Step  5: <H> = 0.332461, overlap = 0.333634, P(ancilla=0) = 0.781800
Step  6: <H> = 0.209711, overlap = 0.389081, P(ancilla=0) = 0.787868
Step  7: <H> = 0.076274, overlap = 0.449365, P(ancilla=0) = 0.796692
Step  8: <H> = -0.065209, overlap = 0.513292, P(ancilla=0) = 0.807636
Step  9: <H> = -0.210799, overlap = 0.579081, P(ancilla=0) = 0.820204
Step 10: <H> = -0.355670, overlap = 0.644549, P(ancilla=0) = 0.833906
Step 11: <H> = -0.494737, overlap = 0.707397, P(ancilla=0) = 0.848214
Step 12: <H> = -0.623373, overlap = 0.765533, P(ancilla=0) = 0.862582
Step 13: <H> = -0.738016, overlap = 0.817346, P(ancilla=0) = 0.876492
Step 14: <H> = -0.836520, overlap = 0.861868, P(ancilla=0) = 0.889510
Step 

/tmp/ipykernel_8376/1104687326.py:35: ComplexWarning: Casting complex values to real discards the imaginary part
  U[:,i] = col / np.linalg.norm(col)


In [224]:
import numpy as np

# 2x2 Hamiltonian
H = np.array([[-1.0, 0.0],
              [ 0.5, 0.0]])
n = H.shape[0]
ancilla = n  # 1 Ancilla → dim = 2n
dim = n + ancilla

# Skalierung auf [-1,1]
alpha = np.max(np.abs(np.linalg.eigvals(H)))
H_scaled = H / alpha

# 1️⃣ Start: U_block 2n×2n, obere n Spalten = H_scaled
U = np.zeros((dim, dim), dtype=complex)
U[:n, :n] = H_scaled

# 2️⃣ Fülle restliche Spalten zufällig
for i in range(n, dim):
    col = np.random.randn(dim) + 1j*np.random.randn(dim)
    # Orthogonal zu bisherigen Spalten
    for j in range(i):
        col -= np.vdot(U[:, j], col) * U[:, j]
    # Normieren
    U[:, i] = col / np.linalg.norm(col)

# 3️⃣ Prüfen Unitarität
print("U_block unitär:", np.allclose(U.conj().T @ U, np.eye(dim)))

# 4️⃣ Reflexion auf Ancilla |0>
R0 = np.diag([1]*n + [-1]*ancilla)

# 5️⃣ Startzustand |0>⊗|+>
psi = np.zeros(dim, dtype=complex)
psi[:n] = 1/np.sqrt(n)

# 6️⃣ QSVT-Iteration
k = 20
for step in range(1, k+1):
    psi = R0 @ U.conj().T @ R0 @ U @ psi

    # Postselection Ancilla=0
    psi_sys = psi[:n]
    prob = np.linalg.norm(psi_sys)**2
    psi_sys /= np.linalg.norm(psi_sys)

    energy = np.vdot(psi_sys, H @ psi_sys).real
    eigvals, eigvecs = np.linalg.eigh(H)
    ground_state = eigvecs[:, 0]
    overlap = np.abs(np.vdot(ground_state, psi_sys))**2

    print(f"Step {step:2d}: <H> = {energy:.6f}, overlap = {overlap:.6f}, P(ancilla=0) = {prob:.6f}")

U_block unitär: False
Step  1: <H> = -1.000000, overlap = 0.853553, P(ancilla=0) = 0.781250
Step  2: <H> = -1.000000, overlap = 0.853553, P(ancilla=0) = 1.535980
Step  3: <H> = -1.000000, overlap = 0.853553, P(ancilla=0) = 1.518209
Step  4: <H> = -1.000000, overlap = 0.853553, P(ancilla=0) = 1.511950
Step  5: <H> = -1.000000, overlap = 0.853553, P(ancilla=0) = 1.507716
Step  6: <H> = -1.000000, overlap = 0.853553, P(ancilla=0) = 1.503849
Step  7: <H> = -1.000000, overlap = 0.853553, P(ancilla=0) = 1.500059
Step  8: <H> = -1.000000, overlap = 0.853553, P(ancilla=0) = 1.496294
Step  9: <H> = -1.000000, overlap = 0.853553, P(ancilla=0) = 1.492545
Step 10: <H> = -1.000000, overlap = 0.853553, P(ancilla=0) = 1.488811
Step 11: <H> = -1.000000, overlap = 0.853553, P(ancilla=0) = 1.485091
Step 12: <H> = -1.000000, overlap = 0.853553, P(ancilla=0) = 1.481385
Step 13: <H> = -1.000000, overlap = 0.853553, P(ancilla=0) = 1.477694
Step 14: <H> = -1.000000, overlap = 0.853553, P(ancilla=0) = 1.47401

In [225]:
import numpy as np
from scipy.linalg import sqrtm

# Hamiltonian
H = np.array([[-1.0, 0.0],
              [ 0.5, 0.0]])
n = H.shape[0]

# Skalierung
alpha = np.max(np.abs(np.linalg.eigvals(H)))
H_scaled = H / alpha
I2 = np.eye(n)

# 1️⃣ Block-Encoding
B = sqrtm(I2 - H_scaled @ H_scaled)  # obere/untere Blocks
U_block = np.block([[H_scaled, B],
                    [B, -H_scaled]])

# Prüfen Unitärität
print("U_block unitär:", np.allclose(U_block.conj().T @ U_block, np.eye(2*n)))

# Startzustand |0>_anc ⊗ |+>_sys
psi = np.zeros(2*n, dtype=complex)
psi[:n] = 1/np.sqrt(n)

# Reflexion auf Ancilla=0
R0 = np.diag([1]*n + [-1]*n)

# Iteration (QSVT)
k = 10
eigvals_full, eigvecs_full = np.linalg.eigh(H)
ground_state = eigvecs_full[:,0]

for step in range(1, k+1):
    psi = R0 @ U_block.conj().T @ R0 @ U_block @ psi

    # Postselection Ancilla=0
    psi_sys = psi[:n]
    prob = np.linalg.norm(psi_sys)**2
    psi_sys /= np.linalg.norm(psi_sys)

    energy = np.vdot(psi_sys, H @ psi_sys).real
    overlap = np.abs(np.vdot(ground_state, psi_sys))**2
    print(f"Step {step:2d}: <H> = {energy:.6f}, overlap = {overlap:.6f}, P(ancilla=0) = {prob:.6f}")

U_block unitär: False
Step  1: <H> = -0.250000, overlap = 0.429289, P(ancilla=0) = 1.250000
Step  2: <H> = -0.041978, overlap = 0.020706, P(ancilla=0) = 0.258708
Step  3: <H> = -0.138004, overlap = 0.313322, P(ancilla=0) = 1.555703
Step  4: <H> = -0.660040, overlap = 0.787294, P(ancilla=0) = 0.146212
Step  5: <H> = -0.242177, overlap = 0.140937, P(ancilla=0) = 0.862009
Step  6: <H> = -1.026021, overlap = 0.999861, P(ancilla=0) = 1.739209
Step  7: <H> = -1.038904, overlap = 0.915245, P(ancilla=0) = 0.142142
Step  8: <H> = -1.027530, overlap = 0.895308, P(ancilla=0) = 3.446728
Step  9: <H> = 0.049805, overlap = 0.062061, P(ancilla=0) = 0.417878
Step 10: <H> = -0.278729, overlap = 0.167045, P(ancilla=0) = 2.475256


/tmp/ipykernel_8376/1544356242.py:15: LinAlgWarning: Matrix is singular. The result might be inaccurate or the array might not have a square root.
  B = sqrtm(I2 - H_scaled @ H_scaled)  # obere/untere Blocks


In [228]:
# Prüfen, ob H Hermitisch ist
is_hermitian = np.allclose(H, H.conj().T)
print("H ist hermitisch:", is_hermitian)

H ist hermitisch: True


In [311]:
import numpy as np
from scipy.linalg import sqrtm

# Hermitisches 2x2 Hamiltonian
H = np.array([[-1.0, 0.5],
              [ 0.5,  0.0]])
n = H.shape[0]

# Skalierung für Block-Encoding
alpha = np.max(np.abs(np.linalg.eigvals(H)))
H_scaled = H / alpha
I = np.eye(n)

# Exaktes Block-Encoding mit 1 Ancilla
B = sqrtm(I - H_scaled @ H_scaled)
U_block = np.block([[H_scaled, B],
                    [B, -H_scaled]])

# Prüfen Unitarität
print("U_block unitär:", np.allclose(U_block.conj().T @ U_block, np.eye(2*n), atol=1e-5))

# Startzustand |0>_anc ⊗ |+>_sys
psi = np.zeros(2*n, dtype=complex)
psi[:n] = 1/np.sqrt(n)

# Reflexion auf Ancilla=0
R0 = np.diag([1]*n + [-1]*n)
def R(phi):
    return np.block([
        [np.exp(1j*phi)*np.eye(n), np.zeros((n,n))],
        [np.zeros((n,n)), np.exp(-1j*phi)*np.eye(n)]
    ])
# QSVT-Iteration
eigvals, eigvecs = np.linalg.eigh(H)
ground_state = eigvecs[:,0]
k = 100

for step in range(1, k+1):
    psi = R(np.pi/2) @ U_block.conj().T @ R(np.pi/2) @ U_block @ psi
    nn=np.linalg.norm(psi)
    psi_sys = psi[:n].copy()
    prob = np.linalg.norm(psi_sys)**2
    psi_sys /= np.linalg.norm(psi_sys)
    energy = np.vdot(psi_sys, H @ psi_sys).real
    overlap = np.abs(np.vdot(ground_state, psi_sys))**2
    print(f"Step {step:2d}: <H> = {energy:.6f}, overlap = {overlap:.6f}, P(ancilla=0) = {prob:.6f}", nn)

U_block unitär: True
Step  1: <H> = -0.022386, overlap = 0.162276, P(ancilla=0) = 0.902454 0.9999999999999993
Step  2: <H> = -0.109374, overlap = 0.223786, P(ancilla=0) = 0.654406 0.9999999999999988
Step  3: <H> = -0.353783, overlap = 0.396609, P(ancilla=0) = 0.369247 0.9999999999999983
Step  4: <H> = -0.960798, overlap = 0.825833, P(ancilla=0) = 0.177332 0.9999999999999978
Step  5: <H> = -1.037590, overlap = 0.880133, P(ancilla=0) = 0.166391 0.9999999999999974
Step  6: <H> = -0.399486, overlap = 0.428926, P(ancilla=0) = 0.341426 0.9999999999999969
Step  7: <H> = -0.125636, overlap = 0.235285, P(ancilla=0) = 0.622423 0.9999999999999964
Step  8: <H> = -0.027994, overlap = 0.166241, P(ancilla=0) = 0.880929 0.9999999999999959
Step  9: <H> = -0.000254, overlap = 0.146627, P(ancilla=0) = 0.998773 0.9999999999999956
Step 10: <H> = -0.017500, overlap = 0.158821, P(ancilla=0) = 0.922085 0.9999999999999949
Step 11: <H> = -0.094833, overlap = 0.213503, P(ancilla=0) = 0.685921 0.9999999999999944


In [229]:
(U_block.conj().T @ U_block)

array([[ 1.00000000e+00+0.00000000e+00j, -3.25180274e-16+2.45172498e-25j,
        -1.21816499e-17-2.54378733e-08j,  2.35521624e-17+1.05367121e-08j],
       [-3.25180274e-16-2.45172498e-25j,  1.00000000e+00+0.00000000e+00j,
         2.00055443e-17+1.05367121e-08j, -1.21816499e-17-4.36444907e-09j],
       [-1.21816499e-17+2.54378733e-08j,  2.00055443e-17-1.05367121e-08j,
         1.00000000e+00+0.00000000e+00j, -3.66861975e-16+2.45172498e-25j],
       [ 2.35521624e-17-1.05367121e-08j, -1.21816499e-17+4.36444907e-09j,
        -3.66861975e-16-2.45172498e-25j,  1.00000000e+00+0.00000000e+00j]])

In [235]:
R0.conj().T@R0

array([[1, 0, 0, 0],
       [0, 1, 0, 0],
       [0, 0, 1, 0],
       [0, 0, 0, 1]])

In [266]:
psi = np.zeros(2*n, dtype=complex)
psi[:n] = 1/np.sqrt(n)


In [296]:
psi = R0 @ U_block.conj().T @ R0 @ U_block @ psi
print(np.linalg.norm(psi))
psi /= np.linalg.norm(psi)
psi_sys = psi[:n].copy()
psi_sys /= np.linalg.norm(psi_sys)
print(np.vdot(psi_sys, H @ psi_sys).real)

0.9999999999999999
-1.1885624762991598


In [297]:
np.linalg.norm(psi)

np.float64(1.0)

In [306]:
R(-np.pi/2)

array([[6.123234e-17-1.j, 0.000000e+00+0.j, 0.000000e+00+0.j,
        0.000000e+00+0.j],
       [0.000000e+00+0.j, 6.123234e-17-1.j, 0.000000e+00+0.j,
        0.000000e+00+0.j],
       [0.000000e+00+0.j, 0.000000e+00+0.j, 6.123234e-17+1.j,
        0.000000e+00+0.j],
       [0.000000e+00+0.j, 0.000000e+00+0.j, 0.000000e+00+0.j,
        6.123234e-17+1.j]])

In [324]:
import numpy as np
from scipy.linalg import sqrtm

# -------------------------------
# 1️⃣ Setup: Hamiltonian & Block-Encoding
# -------------------------------
H = np.array([[-1.0, 0.5],
              [ 0.5,  0.0]])
n = H.shape[0]  # Systemgröße

# Skalierung für Block-Encoding
alpha = np.max(np.abs(np.linalg.eigvals(H)))
H_scaled = H / alpha
I_sys = np.eye(n)

# Exaktes Block-Encoding mit 1 Ancilla
B = sqrtm(I_sys - H_scaled @ H_scaled)
U_block = np.block([[H_scaled, B],
                    [B, -H_scaled]])

# Prüfen Unitarität
print("U_block unitär:", np.allclose(U_block.conj().T @ U_block, np.eye(2*n), atol=1e-12))

# -------------------------------
# 2️⃣ Reflexionen
# -------------------------------

# klassische Ancilla=0 Reflexion
R0 = np.diag([1]*n + [-1]*n)

# Winkel-Reflexion auf Ancilla
def R(phi):
    """R(phi) = exp(-i*phi*Z/2) auf Ancilla, erweitert auf System"""
    R_anc = np.array([[np.exp(-1j*phi/2), 0],
                      [0, np.exp(1j*phi/2)]])
    return np.kron(R_anc, np.eye(n))

# -------------------------------
# 3️⃣ Startzustand: |0>_anc ⊗ |+>_sys
# -------------------------------
psi = np.zeros(2*n, dtype=complex)
psi[:n] = 1/np.sqrt(n)

# Ground state für Overlap-Berechnung
eigvals, eigvecs = np.linalg.eigh(H)
ground_state = eigvecs[:,0]

# -------------------------------
# 4️⃣ QSVT-Iteration
# -------------------------------
k = 20
#phi_list = [0, np.pi] * (k//2)  # feste Winkelfolge

print("\n--- Iteration mit R0 ---")
psi_r0 = psi.copy()
for step in range(1, k+1):
    psi_r0 = R0 @ U_block.conj().T @ R0 @ U_block @ psi_r0
    nn = np.linalg.norm(psi_r0)
    psi_sys = psi_r0[:n].copy()
    prob = np.linalg.norm(psi_sys)**2
    psi_sys /= np.linalg.norm(psi_sys)
    energy = np.vdot(psi_sys, H @ psi_sys).real
    overlap = np.abs(np.vdot(ground_state, psi_sys))**2
    print(f"Step {step:2d}: <H>={energy:.6f}, overlap={overlap:.6f}, P(ancilla=0)={prob:.6f}", nn)

print("\n--- Iteration mit R(phi) ---")
psi_rphi = psi.copy()
for step in range(1, k+1):
    phi = phi_list[step-1]
    Rphi = R(phi)
    psi_rphi = Rphi @ U_block.conj().T @ Rphi @ U_block @ psi_rphi
    nn = np.linalg.norm(psi_rphi)
    psi_sys = psi_rphi[:n].copy()
    prob = np.linalg.norm(psi_sys)**2
    psi_sys /= np.linalg.norm(psi_sys)
    energy = np.vdot(psi_sys, H @ psi_sys).real
    overlap = np.abs(np.vdot(ground_state, psi_sys))**2
    print(f"Step {step:2d}: <H>={energy:.6f}, overlap={overlap:.6f}, P(ancilla=0)={prob:.6f}", nn)

U_block unitär: False

--- Iteration mit R0 ---
Step  1: <H>=-0.022386, overlap=0.162276, P(ancilla=0)=0.902454 0.9999999999999993
Step  2: <H>=-0.109374, overlap=0.223786, P(ancilla=0)=0.654406 0.9999999999999988
Step  3: <H>=-0.353783, overlap=0.396609, P(ancilla=0)=0.369247 0.9999999999999983
Step  4: <H>=-0.960798, overlap=0.825833, P(ancilla=0)=0.177332 0.9999999999999978
Step  5: <H>=-1.037590, overlap=0.880133, P(ancilla=0)=0.166391 0.9999999999999974
Step  6: <H>=-0.399486, overlap=0.428926, P(ancilla=0)=0.341426 0.9999999999999969
Step  7: <H>=-0.125636, overlap=0.235285, P(ancilla=0)=0.622423 0.9999999999999964
Step  8: <H>=-0.027994, overlap=0.166241, P(ancilla=0)=0.880929 0.9999999999999959
Step  9: <H>=-0.000254, overlap=0.146627, P(ancilla=0)=0.998773 0.9999999999999956
Step 10: <H>=-0.017500, overlap=0.158821, P(ancilla=0)=0.922085 0.9999999999999949
Step 11: <H>=-0.094833, overlap=0.213503, P(ancilla=0)=0.685921 0.9999999999999944
Step 12: <H>=-0.312948, overlap=0.36773

In [314]:
import numpy as np

k = 20  # Anzahl Iterationen

# Chebyshev-Polynom-Filter: Arccos-basiert
phis = np.arccos(np.cos((2*np.arange(1, k+1)-1)/(2*k) * np.pi))

# Ausgabe
for i, phi in enumerate(phis, 1):
    print(f"phi_{i:2d} = {phi:.6f} rad, {np.degrees(phi):.2f}°")

phi_ 1 = 0.078540 rad, 4.50°
phi_ 2 = 0.235619 rad, 13.50°
phi_ 3 = 0.392699 rad, 22.50°
phi_ 4 = 0.549779 rad, 31.50°
phi_ 5 = 0.706858 rad, 40.50°
phi_ 6 = 0.863938 rad, 49.50°
phi_ 7 = 1.021018 rad, 58.50°
phi_ 8 = 1.178097 rad, 67.50°
phi_ 9 = 1.335177 rad, 76.50°
phi_10 = 1.492257 rad, 85.50°
phi_11 = 1.649336 rad, 94.50°
phi_12 = 1.806416 rad, 103.50°
phi_13 = 1.963495 rad, 112.50°
phi_14 = 2.120575 rad, 121.50°
phi_15 = 2.277655 rad, 130.50°
phi_16 = 2.434734 rad, 139.50°
phi_17 = 2.591814 rad, 148.50°
phi_18 = 2.748894 rad, 157.50°
phi_19 = 2.905973 rad, 166.50°
phi_20 = 3.063053 rad, 175.50°


In [316]:
print(phi_list),phis

[0, 3.141592653589793, 0, 3.141592653589793, 0, 3.141592653589793, 0, 3.141592653589793, 0, 3.141592653589793, 0, 3.141592653589793, 0, 3.141592653589793, 0, 3.141592653589793, 0, 3.141592653589793, 0, 3.141592653589793]


(None,
 array([0.07853982, 0.23561945, 0.39269908, 0.54977871, 0.70685835,
        0.86393798, 1.02101761, 1.17809725, 1.33517688, 1.49225651,
        1.64933614, 1.80641578, 1.96349541, 2.12057504, 2.27765467,
        2.43473431, 2.59181394, 2.74889357, 2.9059732 , 3.06305284]))

In [321]:
phi_list = list(enumerate(phis,1))

In [322]:
phi_list = [p for _,p in phi_list]

In [323]:
phi_list

[np.float64(0.078539816339745),
 np.float64(0.23561944901923468),
 np.float64(0.3926990816987242),
 np.float64(0.5497787143782139),
 np.float64(0.7068583470577035),
 np.float64(0.8639379797371933),
 np.float64(1.0210176124166828),
 np.float64(1.1780972450961724),
 np.float64(1.335176877775662),
 np.float64(1.4922565104551517),
 np.float64(1.6493361431346414),
 np.float64(1.8064157758141308),
 np.float64(1.9634954084936207),
 np.float64(2.1205750411731104),
 np.float64(2.2776546738526),
 np.float64(2.4347343065320897),
 np.float64(2.5918139392115793),
 np.float64(2.748893571891069),
 np.float64(2.9059732045705586),
 np.float64(3.0630528372500483)]

In [325]:
import numpy as np
from scipy.linalg import sqrtm

# -------------------------------
# Hamiltonian & Block-Encoding
# -------------------------------
H = np.array([[-1.0, 0.5],
              [ 0.5,  0.0]])
n = H.shape[0]

alpha = np.max(np.abs(np.linalg.eigvals(H)))
H_scaled = H / alpha
I_sys = np.eye(n)

B = sqrtm(I_sys - H_scaled @ H_scaled)
U_block = np.block([[H_scaled, B],
                    [B, -H_scaled]])

# Startzustand
psi = np.zeros(2*n, dtype=complex)
psi[:n] = 1/np.sqrt(n)

# Reflexion R(phi)
def R(phi):
    R_anc = np.array([[np.exp(-1j*phi/2), 0],
                      [0, np.exp(1j*phi/2)]])
    return np.kron(R_anc, np.eye(n))

# Ground state
eigvals, eigvecs = np.linalg.eigh(H)
ground_state = eigvecs[:,0]

# -------------------------------
# QSVT-Phasen optimiert
# -------------------------------
k = 20
epsilon = 0.1  # weiche Anpassung
phis = np.arccos((1-epsilon) * np.cos((2*np.arange(1, k+1)-1)/(2*k) * np.pi))

# -------------------------------
# Iteration
# -------------------------------
psi_rphi = psi.copy()
print("\n--- Optimierte QSVT-Iteration ---")
for step in range(1, k+1):
    phi = phis[step-1]
    Rphi = R(phi)
    psi_rphi = Rphi @ U_block.conj().T @ Rphi @ U_block @ psi_rphi
    nn = np.linalg.norm(psi_rphi)
    psi_sys = psi_rphi[:n].copy()
    prob = np.linalg.norm(psi_sys)**2
    psi_sys /= np.linalg.norm(psi_sys)
    energy = np.vdot(psi_sys, H @ psi_sys).real
    overlap = np.abs(np.vdot(ground_state, psi_sys))**2
    print(f"Step {step:2d}: <H>={energy:.6f}, overlap={overlap:.6f}, P(ancilla=0)={prob:.6f}", nn)


--- Optimierte QSVT-Iteration ---
Step  1: <H>=-0.001043, overlap=0.147184, P(ancilla=0)=0.994987 0.9999999999999996
Step  2: <H>=-0.004661, overlap=0.149742, P(ancilla=0)=0.977990 0.9999999999999992
Step  3: <H>=-0.012338, overlap=0.155171, P(ancilla=0)=0.943775 0.999999999999999
Step  4: <H>=-0.026796, overlap=0.165394, P(ancilla=0)=0.885441 0.9999999999999983
Step  5: <H>=-0.052819, overlap=0.183796, P(ancilla=0)=0.796791 0.9999999999999976
Step  6: <H>=-0.099439, overlap=0.216760, P(ancilla=0)=0.675615 0.9999999999999971
Step  7: <H>=-0.185364, overlap=0.277519, P(ancilla=0)=0.527700 0.9999999999999966
Step  8: <H>=-0.352067, overlap=0.395395, P(ancilla=0)=0.370380 0.9999999999999957
Step  9: <H>=-0.679932, overlap=0.627231, P(ancilla=0)=0.233481 0.9999999999999956
Step 10: <H>=-1.129071, overlap=0.944821, P(ancilla=0)=0.154999 0.9999999999999947
Step 11: <H>=-1.011476, overlap=0.861668, P(ancilla=0)=0.169957 0.9999999999999943
Step 12: <H>=-0.497349, overlap=0.498125, P(ancilla=0